In [6]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1
)

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content = "You are a geography expert. And you only reply in Italian."
    ), 
    AIMessage(
        content="Ciao, mi chiamo Paolo!"
    ),
    HumanMessage(
        content="What is the distance between Mexico and Thailand. And what is your name?"
    )
]